The following is my initial EDA for the Gendered Pronoun Resolution dataset from the similarly named competition. I'm presently working with a group to make a late attempt to see if we can find a better solution to the ones which were submitted. This kernal is meant as a starting point for my group and is far from a finished end product.

Version 2: removed pronouns from the stopwords list as they are relevent to this challenge. added the side effect of the word 'Hi' showing up and being counted frequently, which is not present in the prior version, you can see the impact side by side

In [ ]:
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import sent_tokenize, word_tokenize
import nltk.data

In [ ]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
stage_one = pd.read_csv('../input/gendered-pronoun-resolution/test_stage_1.tsv', sep='\t')
stage_two = pd.read_csv('../input/gendered-pronoun-resolution/test_stage_2.tsv', sep='\t')

In [ ]:
display(stage_one.head())
display(stage_two.head())

In [ ]:
print(stage_one.Text.iloc[0])


In [ ]:
stng = stage_one.Text.iloc[0]
print(stng[191:204]) # The A-offset or where the Subject is
print(stng[207:211]) # The B-offset or where the object is
print(stng[274:278]) # The location of the pronoun

In [ ]:
print(stage_two.Text.iloc[3])

In [ ]:
stage_one.shape

In [ ]:
stage_one['length'] = pd.Series([len(x) for x in stage_one.Text], index=stage_one.index)
stage_two['length'] = pd.Series([len(x) for x in stage_two.Text], index=stage_two.index)


In [ ]:
print('Stage one data: ')



display(stage_one.describe())
display(stage_one.shape)
pronoun = stage_one.groupby("Pronoun")
print('\n\npronoun data: ')
display(pronoun.describe())
#display(pronoun.length)

plt.figure(figsize=(20,7))
sns.distplot(stage_one.length)
plt.xlabel("Frequency")
plt.ylabel("length of of text")


plt.figure(figsize=(12,7))
pronoun.size().sort_values().plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Pronoun")
plt.ylabel("Frequency of pronoun")
plt.show()

In [ ]:
print('Stage two data: ')
display(stage_two.describe())
display(stage_two.shape)
pronoun = stage_two.groupby("Pronoun")
print('pronoun data: ')
display(pronoun.describe())

plt.figure(figsize=(20,7))
sns.distplot(stage_two.length, color='g')
plt.xlabel("Frequency")
plt.ylabel("length of text")

plt.figure(figsize=(12,7))
pronoun.size().sort_values().plot.bar(color='g')
plt.xticks(rotation=50)
plt.xlabel("Pronoun")
plt.ylabel("Frequency of pronoun")
plt.show()

# Let's make some MF word clouds!
Full discloser I have never made a word cloud before, this project seemed like a good time to give it a try.
The link below is what I used for reference material and also some copying of a few lines of code.

https://www.datacamp.com/community/tutorials/wordcloud-python

I highly recommend that tutorial there are some gorgeos wordplots there which exceed the scope of this perticular notebook

In [ ]:

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
# Credit to DJ here, she had mentioned that pronouns can show as stop words (words that NLP tools will ignore as inconsequential)
# This cell collects the recognized pronouns and adds them to a list
non_stop = set(pronoun.Pronoun.unique().index)
print(non_stop)

non_stop = STOPWORDS - non_stop

In [ ]:
#?WordCloud
#uncomment to see documentation for wordcloud, I had to reference this one alot

In [ ]:
text = " ".join(sample for sample in stage_one.Text)
print("There are ", len(text)," words in the combination of all strings from 'Text' column.")


wordcloud = WordCloud(stopwords=STOPWORDS,width=1200,
    height=600).generate(text)

plt.figure(figsize=(20,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('WordCloud without pronouns', fontsize=26)
plt.axis("off")
plt.show()



wordcloud = WordCloud(stopwords=non_stop,width=1200,
    height=600).generate(text)

plt.figure(figsize=(20,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('WordCloud with pronouns', fontsize=26)
plt.axis("off")
plt.show()

In [ ]:
text = " ".join(sample for sample in stage_two.Text)
print("There are ", len(text)," words in the combination of all strings from 'Text' column.")
print("this is 5x the number of words in the stage one dataset")
wordcloud = WordCloud(stopwords=STOPWORDS,width=1200,
    height=600).generate(text)

plt.figure(figsize=(20,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('WordCloud without pronouns', fontsize=26)
plt.axis("off")
plt.show()


wordcloud = WordCloud(stopwords=non_stop,width=1200,
    height=600).generate(text)

plt.figure(figsize=(20,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('WordCloud with pronouns', fontsize=26)
plt.axis("off")
plt.show()

Further consideration: It might be interesting to break down the text samples differently, for instance spliting the strings into two to four word chunks and looking for common chunks (words occuring frequently in the same sequence).

In [ ]:
q = stage_one.Text[0]
#q = q.split()
#sentences = ["[CLS] " + val + " [SEP]" for val in q.split('.')]
print(cleaned string)